In [1]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import time
import re
from tqdm import tqdm
import timeit
tqdm.pandas()
import omap


In [38]:
from datetime import datetime

ts = int("1573750800")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ts = int("1603729564")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ts = int("1519603200")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))


2019-11-14 17:00:00
2020-10-26 16:26:04
2018-02-26 00:00:00


In [1]:
# input_folder_path =  './EVR_T23_Car46_261020/'
# #input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# start_time = '2020/10/26 15:50:00' 
# end_time = '2020/10/26 16:50:00'
# #start_time = '2020/10/26 04:36:45'
# #end_time = '2020/10/26 04:37:09'
# sample_output_filename = ['./EVR_T23_Car46_261020/preprocessing_output/EVR_20201026_1550_to_20201026_1650.csv']

input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
#input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
start_time = '2019/11/14 17:00:00' 
end_time = '2019/11/14 18:00:00'
sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_1800.csv']

# input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# #input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# start_time = '2019/11/14 18:00:00' 
# end_time = '2019/11/14 19:00:00'
# sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1800_to_20191114_1900.csv']

def process_evr (input_folder_path, start_time, end_time):
    # Determine time difference between start and end time
    datetime_format = '%Y/%m/%d %H:%M:%S'
    start_time = datetime.datetime.strptime(start_time, datetime_format)
    end_time = datetime.datetime.strptime(end_time, datetime_format)
    start_time_new = start_time.strftime('%m/%d/%Y %H:%M:%S')
    end_time_new = end_time.strftime('%m/%d/%Y %H:%M:%S')
    # Check if EVR folder exist
    if not os.path.exists(os.path.join(input_folder_path, 'EVR')):
        print("EVR folder does not exist. Kindly check the folder strucutre. No EVR processing will be performed")
    else:
        print("EVR folder exist!")
        # Check if C2 file exist
        c2_pattern = 'EVR_Car\d+_\d+_C2.txt'
        norm_pattern = 'EVR_Car\d+_\d+.txt'
        c2_flag = False
        norm_flag = False
        for evr_file in os.listdir(os.path.join(input_folder_path, 'EVR')):
            if re.search(c2_pattern, evr_file):
                c2_filename = evr_file
                c2_flag = True
            if re.search(norm_pattern,evr_file):
                norm_filename = evr_file
                norm_flag = True

        if not (c2_flag and norm_flag):
            sys.exit("Missing EVR files! Kindly check EVR folder!")
        else:
            # Process C2 file
            df_c2 = pd.read_csv(os.path.join(input_folder_path, 'EVR', c2_filename), sep=";")
            df_c2['Date'] = pd.to_datetime(df_c2['Date'], format='%m/%d/%Y %H:%M:%S')
            df_c2_period = df_c2.loc[(df_c2['Date'] >= start_time_new) & (df_c2['Date'] <= end_time_new)]
            #df_c2_clean_timestamp = preprocess_timestamp_old(df_c2_period)
            df_c2_clean = clean_dataframe(df_c2_period, 'C2')
            #df_c2_clean.to_csv('./result_c2.csv', index=False, header=True)
            
            # Processs Operating Data file
            df_od = pd.read_csv(os.path.join(input_folder_path, 'EVR', norm_filename), sep=";")
            df_od['Date'] = pd.to_datetime(df_od['Date'], format='%m/%d/%Y %H:%M:%S')
            df_od_period = df_od.loc[(df_od['Date'] >= start_time_new) & (df_od['Date'] <= end_time_new)]
            #df_od_clean_timestamp = preprocess_timestamp_old(df_od_period)
            df_od_clean = clean_dataframe(df_od_period, 'OD')
            #df_od_clean.to_csv('./result_od.csv', index=False, header=True)
            
            # Merge other Operating Data to C2 Dataframe based on Timestampms (ordered manner)
            df_result = pd.merge_ordered(df_c2_clean, df_od_clean, how='outer', on='Timestampms')
            #df_result.to_csv('./result_1.csv', index=False, header=True)
            
            
            df_result = process_timestamp(df_result, df_c2_clean, df_od_clean, start_time, end_time)
            #df_result.to_csv('./result_9.csv', index=False, header=True)
            # Merge C2 and OD files first
            
            # Remove similar record numbers for both EC2 and EOD
            #df_duplicate = df[df.duplicated('EC2_Record Number',keep='first')]
            filter_col = [col for col in df_result if col.startswith('EC2_')]
            df_result.loc[df_result.duplicated('EC2_Record Number',keep='first'), filter_col] = np.NaN
            filter_col = [col for col in df_result if col.startswith('EOD_')]
            df_result.loc[df_result.duplicated('EOD_Record Number',keep='first'), filter_col] = np.NaN
            
            # Group resulting same real timestamp together
            df_result = df_result.groupby("Timestampms").last().reset_index()

            #df_result.to_csv('./result_8.csv', index=False, header=True)
            # Clean Timestamp
            #df_result = preprocess_timestamp(df_result)
            
            
    return df_c2_clean, df_od_clean, df_result


df_c2_clean, df_od_clean, df_result = process_evr(input_folder_path, start_time, end_time)

output(sample_output_filename[0], df_result)
#unit_test(sample_output_filename[0], 'C2', df_c2_clean)  
#unit_test(sample_output_filename[0], 'OD', df_od_clean)  
#unit_test_all(sample_output_filename[0], df_result)

NameError: name 'datetime' is not defined

In [4]:
def clean_dataframe(df, name):
    # Drop columns with NaNs
    df = df.dropna(how='any',axis=1)
    # Sort df based on column 'Date'
    df = df.sort_values(by='Date',ascending=True).reset_index(drop=True)
    # Convert datetime to timestamp (Unix) in milisecond resolution
    df['Timestamp'] = df.Date.values.astype(np.int64) // 10 ** 9 *1000
    # Add column for Real_Time Timestampms
    df['Timestampms'] = df['Timestamp'] 
    
    # Add Prefix to Columns name (e.g. EC2/EOD_***) except Timestampms
    if name == 'C2':
        df.columns = df.columns.map(lambda x : 'EC2_'+x if x !='Timestampms' else x)
    elif name == 'OD':
        df.columns = df.columns.map(lambda x : 'EOD_'+x if x !='Timestampms' else x)
                
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]    
    return df

In [1]:
def process_timestamp(df, df_c2_clean, df_od_clean, start_time, end_time):
    # Find duplicates datetime in column 'Timestampms'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
    previous_datetime = 0
    consec_flag = False
    for index, unique_datetime in enumerate (df_duplicate['Timestampms'].unique()):
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
              
        # Find difference from previous and next timestamp
        difference_previous,  difference_next = find_difference(timestamp_df, unique_datetime, df)
        
        #Check if record number is in duplicate
        df_ec_record_number = timestamp_df[timestamp_df.duplicated('EC2_Record Number',keep=False)]
        df_od_record_number = timestamp_df[timestamp_df.duplicated('EOD_Record Number',keep=False)]
        
        # EC2 Data has duplciate timestamp
        if len(df_ec_record_number) == 0 and len(df_od_record_number) > 0:
            df = split_equally(timestamp_df, df, 'C2')
            
        # OD Data has duplicate timestamp
        elif len(df_ec_record_number) > 0 and len(df_od_record_number) == 0:
            df_od_timestamp = df_od_clean.loc[df_od_clean['Timestampms'] == unique_datetime]
            df_od_timestamp_next = df_od_clean.loc[df_od_clean['Timestampms'] == df_duplicate['Timestampms'].unique()[index+1]]
                        
            od_difference_previous,  od_difference_next = find_difference(df_od_timestamp, unique_datetime, df_od_clean)
            
            handled_previous =  df.loc[timestamp_df.index[0]-1,'Timestampms'] - df.loc[timestamp_df.index[0]-1,'EOD_Timestamp']
            print(consec_flag)
            
            if difference_previous < 1000 or df.loc[timestamp_df.index[0],'EOD_Record Number'] > df.loc[timestamp_df.index[-1],'EOD_Record Number']:
                df = split_equally(timestamp_df, df, 'OD')    
            
            elif df.loc[timestamp_df.index[0],'EOD_Record Number'] < df.loc[timestamp_df.index[-1],'EOD_Record Number']:
                #print(df_od_clean.loc[df_od_timestamp.index[0],'Timestampms'],  df.loc[timestamp_df.index[0]-1,'Timestampms'])
                if difference_previous < 1000:
                    df = split_equally(timestamp_df, df, 'OD')
                    
                elif od_difference_previous == 1000 and od_difference_next == 1000:
                    df = split_equally(timestamp_df, df, 'OD')
                    
                elif df_od_clean.loc[df_od_timestamp.index[0],'Timestampms'] - df.loc[timestamp_df.index[0]-1,'Timestampms'] == 1000 and (od_difference_previous >= 2000 and od_difference_next == 1000) and consec_flag:
                    df = split_equally(timestamp_df, df, 'OD')
                    
                elif difference_previous == 1000 and od_difference_previous >= 2000 and od_difference_next == 1000 and not consec_flag:
                    df = split_backward(timestamp_df, df, 'OD')
                    
                else:
                    df = split_forward(timestamp_df, df, 'OD')
                    
                    
            if not df_od_timestamp_next.empty and (df_od_timestamp_next.index[0]-df_od_timestamp.index[-1]==1):
                consec_flag = True
            else:
                consec_flag = False
            
        previous_datetime = unique_datetime
            
    return df


In [50]:
def find_difference(timestamp_df, unique_datetime, df):
    difference_previous, difference_next = 0 , 0
    if timestamp_df.index[0]-1 >=0 and timestamp_df.index[-1]+1 <= len(df)-1:
        difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
        difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
    elif timestamp_df.index[-1] == len(df)-1:
        difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
    elif timestamp_df.index[0] == 0:
        difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
    
    return difference_previous, difference_next

def split_equally(timestamp_df, df, name):
    print("--- " + name +" Handling Duplicates, Split duplicate evenly---")
    # For every Timestamp (1000 millisecond), find the interval
    interval_ms = round(1000 / len(timestamp_df.index))
    for counter in range (0, len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
        print(df.loc[timestamp_df.index[counter],'E'+name+'_Record Number'], df.loc[timestamp_df.index[counter],'E'+name+'_Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'E'+name+'_Record Number'])
    return df

def split_forward(timestamp_df, df, name):
    print("---Handling Duplicates, Fill gap with duplicate, Forward---")
    for counter in range (0, len(timestamp_df.index)):
        df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*1000
        print(df.loc[timestamp_df.index[counter],'E'+name+'_Record Number'], df.loc[timestamp_df.index[counter],'E'+name+'_Date'],
              df.loc[timestamp_df.index[counter],'Timestampms'],df.loc[timestamp_df.index[counter],'E'+name+'_Record Number'])
    return df

def split_backward(timestamp_df, df, name):
    # Loop backwards
    print("---Handling Duplicates, Fill gap with duplicate, Backward---")
    count = 0
    for counter in range (len(timestamp_df.index), 0, -1):
        df.loc[timestamp_df.index[counter-1],'Timestampms'] = df.loc[timestamp_df.index[counter-1],'Timestampms'] - count*1000
        count +=1
        print(df.loc[timestamp_df.index[counter-1],'E'+name+'_Record Number'], df.loc[timestamp_df.index[counter-1],'E'+name+'_Date'],
              df.loc[timestamp_df.index[counter-1],'Timestampms'],df.loc[timestamp_df.index[counter-1],'E'+name+'_Record Number'])
    
    return df

def output(sample_output_filename, df_result):
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    
    df_combine = pd.DataFrame()
    df_combine = df_combine.assign(epoch = df_output['epoch'])
    df_combine = df_combine.assign(Timestampms = df_result['Timestampms']) 
    df_combine = df_combine.assign(EC2_Record_Number = df_result['EC2_Record Number']) 
    df_combine = df_combine.assign(EC2_Date = df_result['EC2_Date']) 
    df_combine = df_combine.assign(EOD_Record_Number = df_result['EOD_Record Number']) 
    df_combine = df_combine.assign(EOD_Date = df_result['EOD_Date']) 

    
    df_combine.to_csv('./result_combine.csv', index=False, header=True)
    
def unit_test(sample_output_filename, name, df_test):
    print("---Unit Test for " + name + " Dataframe---")
    # Unit Test for C2
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    # Only retrieve respective columns
    if name == 'C2':
        df_output = df_output.drop(df_output[(df_output['EC2_002_Record_Number'].isnull())].index)
        #drop column with prefix EOD
        df_output = df_output.loc[:, ~df_output.columns.str.startswith('EOD')]
    elif name == 'OD':
        df_output = df_output.drop(df_output[(df_output['EOD_002_Record_Number'].isnull())].index)
        #drop column with prefix EOD
        df_output = df_output.loc[:, ~df_output.columns.str.startswith('EC2')]
    
    df_output['epoch'] = df_output.epoch.values.astype(np.float64)
    df_output = df_output.reset_index(drop=True)
    print(df_output.shape, df_test.shape)    
    #Output to CSV
    #df_output.to_csv('./df_output.csv', index=False, header=True)

    #Output to CSV
    #df_test.to_csv('./df_test.csv', index=False, header=True)
    #df_drop_test = pd.read_csv('./df_test.csv')
    df_drop_test = df_test
    df_test['result'] = np.where(df_test['Timestampms'] == df_output['epoch'], 'True', 'False')
    df_test.to_csv('./df_test.csv', index=False, header=True)
    #df_drop_test = df_drop_test.sort_values(by='ATO_Real_Timestampms',ascending=True).reset_index(drop=True)
    # Assert whether sample output and self processed are equal
    assert_equal = omap.nan_equal(df_drop_test['Timestampms'].values, df_output['epoch'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    assert_equal = omap.nan_equal(df_drop_test['E'+name+'_Record Number'].values, df_output['E'+name+'_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    
    #df_drop_test.columns = df_output.columns
    #print(np.testing.assert_allclose(df_drop_test.values, df_output.values, rtol=1e-10, atol=0))
    #print(pd.testing.assert_frame_equal(df_drop_test, df_output, check_dtype=False))
    #print(df_drop_test.compare(df_output, align_axis=0))
    #assert_equal = nan_equal(df_drop_test.values, df_output.values)
    #assert_equal = nan_equal(df_drop['ATO_* General'].values, df_output['ATO_0101__General'].values)
    #print("Equality Between Sample Output and Self Processed: ", assert_equal)
    #print(np.testing.assert_equal(df_drop_test.values, df_output.values))
    
def unit_test_all(sample_output_filename, df_test):
    print("---Unit Test for entire Dataframe---")
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    print(df_output.shape, df_test.shape)
    assert_equal = omap.nan_equal(df_test['Timestampms'].values, df_output['epoch'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    df_test['result'] = np.where(df_test['Timestampms'] == df_output['epoch'], 'True', 'False')
    #df_test.to_csv('./df_test.csv', index=False, header=True)
        
    df_test['result_1'] = np.where(df_test['EC2_Record Number'] == df_output['EC2_002_Record_Number'], 'True', 'False')
    df_test.to_csv('./df_test.csv', index=False, header=True)
    assert_equal = omap.nan_equal(df_test['EC2_Record Number'].values, df_output['EC2_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    assert_equal = omap.nan_equal(df_test['EOD_Record Number'].values, df_output['EOD_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    print(np.testing.assert_equal(df_test['EC2_Record Number'].values, df_output['EC2_002_Record_Number'].values))